<a href="https://colab.research.google.com/github/itallonardi/bairrosEDistritosdoBrasil/blob/main/bairrosEDistritosDoBrasil.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extração de dados para criação de datasets de bairros, distritos e povoados do Brasil

Este notebook realiza a coleta de dados de bairros, distritos e povoados do Brasil utilizando uma abordagem combinada entre a API do IBGE e OpenStreetMap, e bibliotecas de geoprocessamento como `osmnx` e `geopandas` para driblar a falta de dados oficiais de qualidade para bairros, distritos e povoados dos municípios brasileiros.

## Entendendo o problema

Uma extração de dados utilizando a API do SIDRA, do IBGE, retorna cerca de 14.320 bairros e distritos para um total de 5.570 municípios. Ou seja, apenas 2,57 bairros por cidade. Esta abordagem mista, por outro lado, conseguiu extrair um número significativamente maior de bairros, povoados e distritos, totalizando 46.350 bairros, povoados e distritos (uma média de 8,32 bairros por municípios, 3,23x mais que o retornado pela API SIDRA), proporcionando uma visão mais abrangente das áreas urbanas e rurais no Brasil.

### Instalando as bibliotecas osmnx e geopandas:

In [1]:
pip install osmnx geopandas requests pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.2/107.2 kB 3.6 MB/s eta 0:00:00


### Importação e configuração:

In [2]:
import osmnx as ox
import geopandas as gpd
import requests
import pandas as pd
import os

ox.settings.use_cache = False
ox.settings.log_console = False

### Definição das Constantes e Funções Auxiliares:
Aqui temos a definição dos estados brasileiros e do Distrito Federal, além das fuções para obter as cidades, cidades por estado, bairros, distritos e povoados e, adicionalmente, uma consulta a tabela de dados agregados do IBGE (série 2010, a mais recente) para determinar o percentual de pessoas residentes na zona urbana e o percentual residente na zona rural.

In [4]:
VALID_UFS = [
    'AC', 'AL', 'AM', 'AP', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MG', 'MS', 'MT', 'PA', 'PB', 'PE', 'PI', 'PR', 'RJ', 'RN', 'RO', 'RR', 'RS', 'SC', 'SE', 'SP', 'TO'
]


def get_all_cities():
    url = "https://servicodados.ibge.gov.br/api/v1/localidades/municipios"
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.json()
    except requests.RequestException as e:
        print(f"Erro na requisição: {e}")
        return None


def get_population_data(city_code, city, state):
    url = f"https://servicodados.ibge.gov.br/api/v3/agregados/761/periodos/-6/variaveis/1000093?localidades=N6[{city_code}]&classificacao=387[9693,9695]"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        urban_percentage = rural_percentage = None
        for result in data[0]['resultados']:
            for series in result['series']:
                if "9693" in result['classificacoes'][0]['categoria']:
                    urban_percentage = series['serie'].get('2010')
                if "9695" in result['classificacoes'][0]['categoria']:
                    rural_percentage = series['serie'].get('2010')
        print(f"Informaçẽs de proporção rural/urbana da população obtidas para {city}, {state}: {urban_percentage}% em zona urbana, {rural_percentage}% em zona rural.")
        return urban_percentage, rural_percentage
    except requests.RequestException as e:
        print(f"Erro na requisição: {e}")
        return None, None


def get_neighborhoods(query):
    try:
        neighborhoods = ox.features_from_place(query, tags={'place': 'suburb'})
        if neighborhoods.empty:
            print(f"Nenhum bairro encontrado para {query}")
            return gpd.GeoDataFrame()
        print(f"Bairros obtidos para {query}:\n{neighborhoods}")
        return neighborhoods
    except (TypeError, ox._errors.InsufficientResponseError) as e:
        print(f"Erro ao obter bairros para {query}: {e}")
        return gpd.GeoDataFrame()


def get_villages(query):
    try:
        villages = ox.features_from_place(query, tags={'place': ['village']})
        if villages.empty:
            print(f"Nenhum povoado encontrado para {query}")
            return gpd.GeoDataFrame()
        print(f"Povoados e/ou distritos obtidos para {query}:\n{villages}")
        return villages
    except (TypeError, ox._errors.InsufficientResponseError) as e:
        print(f"Erro ao obter povoados para {query}: {e}")
        return gpd.GeoDataFrame()


def get_filtered_cities(cities_data, state_filter, city_filter):
    filtered_cities = []
    for city_info in cities_data:
        state = city_info['microrregiao']['mesorregiao']['UF']['sigla']
        city = city_info['nome']

        if (state_filter == '*' or state.lower() == state_filter.lower()) and \
           (city_filter == '*' or city.lower() == city_filter.lower()):
            filtered_cities.append(city_info)

    return filtered_cities

def combine_all_state_files(output_dir='csv'):
    import glob
    import pandas as pd

    all_csv_files = glob.glob(os.path.join(output_dir, '*.csv'))

    if not all_csv_files:
        print("Nenhum arquivo CSV encontrado para combinar.")
        return


    combined_df = pd.concat([pd.read_csv(file) for file in all_csv_files])

    combined_output_file = os.path.join(output_dir, 'data_BR_*.csv')
    combined_df.to_csv(combined_output_file, index=False)
    print(f"Arquivo combinado para todo o Brasil salvo em {combined_output_file}")

### Função de Processamento e Função Principal

In [ ]:
def process_state_and_city(state_filter, city_filter):
    cities_data = get_all_cities()
    if not cities_data:
        print("Erro ao obter dados das cidades.")
        return

    filtered_cities = get_filtered_cities(
        cities_data, state_filter, city_filter)
    data_list = []

    for city_info in filtered_cities:
        city = city_info['nome']
        state = city_info['microrregiao']['mesorregiao']['UF']['sigla']
        city_code = city_info['id']

        urban_percentage, rural_percentage = get_population_data(
            city_code, city, state)

        if state.lower() == 'df':
            query = 'Distrito Federal, Brazil'
        else:
            query = f"{city}, {state}, Brazil"

        neighborhoods = get_neighborhoods(query)
        villages = get_villages(query)

        if neighborhoods.empty and villages.empty:
            print(f"Sem dados de bairros ou povoados para {city}, {state}.")
            continue

        if not neighborhoods.empty:
            for idx, row in neighborhoods.iterrows():
                location_name = row['name'] if 'name' in row and row['name'] else 'Não identificado'
                data_list.append({
                    'city': city,
                    'state': state,
                    'urban_population': urban_percentage,
                    'rural_population': rural_percentage,
                    'location_name': location_name,
                    'type': 'neighborhood',
                    'geometry': row['geometry'].wkt
                })

        if not villages.empty:
            for idx, row in villages.iterrows():
                location_name = row['name'] if 'name' in row and row['name'] else 'Não identificado'
                data_list.append({
                    'city': city,
                    'state': state,
                    'urban_population': urban_percentage,
                    'rural_population': rural_percentage,
                    'location_name': location_name,
                    'type': 'district',
                    'geometry': row['geometry'].wkt
                })

    df = pd.DataFrame(data_list)

    output_dir = 'csv'
    os.makedirs(output_dir, exist_ok=True)

    output_file = os.path.join(output_dir, f"data_BR_{state_filter}_{city_filter}.csv")

    df.to_csv(output_file, index=False)
    print(f"Dados salvos em {output_file}")
    return output_file


def main(state_filter='*', city_filter='*'):
    if state_filter == '*':
        all_files = []
        for idx, state in enumerate(VALID_UFS, start=1):
            print(f"Processando o estado {idx} de {len(VALID_UFS)}: {state}")
            output_file = process_state_and_city(state, city_filter)
            all_files.append(output_file)

        combine_all_state_files()
    else:
        process_state_and_city(state_filter, city_filter)

### Funções para Entrada do Usuário

In [ ]:
def get_valid_uf():
    while True:
        uf = input("Digite a sigla do estado (ou * para todos): ").strip().upper()
        if uf in VALID_UFS or uf == '*':
            return uf
        else:
            print("UF inválida. Digite uma sigla válida ou * para todos.")


def get_valid_city(state_filter):
    if state_filter.lower() == 'df':
        return '*'
    while True:
        city = input("Digite o nome da cidade (ou * para todas): ").strip()
        if city:
            return city
        else:
            print("Nome da cidade não pode ser vazio.")



### Execução do Script

Neste ponto, a execução do script irá solicitar que você forneça uma UF para extrair os dados. Você pode especificar * (asterisco) para obter todas. Isso irá obter dados de todos os estados e municípios do Brasil, estado a estado, incluindo a geração de um arquivo csv (dentro da pasta csv) para cada estado e um arquivo csv com dados de todo o Brasil.

Se, em vez disso, você informar uma UF, então um novo input surgirá solicitando o nome da cidade. Aqui, outra vez, você pode informar *, neste caso, para todas as cidades do respectivo estado que você selecionou anteriormente, ou especificando o nome da cidade que você deseja.

Os datasets gerados serão salvos na pasta `csv` com o padrão de nomenclatura `data_BR_UF_cidade.csv`, onde `UF` e `cidade` podem ser substituídos pela sigla do estado e o nome da cidade, respectivamente, ou `*` representando todos. Por exemplo:
- `data_BR_BA_*.csv` (todas as cidades da Bahia)
- `data_BR_*.csv` (todos os estados e cidades do Brasil)

In [ ]:
state_filter = get_valid_uf()
city_filter = get_valid_city(state_filter)

main(state_filter, city_filter)

### Execução do Script

Se desejar criar um arquivo único para todo o Brasil, execute o código abaixo:

In [5]:
combine_all_state_files()

Arquivo combinado para todo o Brasil salvo em csv/data_BR_*.csv
